In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

In [3]:
url = "https://www.numbeo.com/cost-of-living/"   # URL for collecting names of the countries from the main page

url_country = "https://www.numbeo.com/cost-of-living/country_result.jsp"  # URL for collecting names of cities in those countries

url_city = "https://www.numbeo.com/cost-of-living/in/"   # URL for collecting data in collected cities

In [4]:
page = requests.get(url)
print(page)   # <Response [200]>

<Response [200]>


'https://www.numbeo.com/cost-of-living/'

In [5]:
html = BeautifulSoup(page.text, 'html')   # Parse html code 

In [6]:
columns = html.find_all('td')[2:7]   # Getting 5 columns with country names from numbeo main page

In [7]:
countries = []   # List for storing countries
for column in columns:
    countries_list = column.find_all('a')   # list of countries in each column
    for country in countries_list:
        countries.append(country.text)   # Filling the list with countries

In [8]:
print(len(countries))   # 235 countries in 5 columns

235


In [9]:
param = {}
param["displayCurrency"] = "USD"   # Parameter for displaying all prices in USD
param["country"] = countries[0]    # Parameter for accessing country page
page_country = requests.get(url_country, params=param)
html = BeautifulSoup(page_country.text, 'html')   # Parse html code 

In [10]:
table = html.find_all('table')[1]   # Getting the main table with all prices

In [12]:
rows_headers = table.find_all('tr')   # Getting each row in that table

In [13]:
titles = ["City", "Country", "URL", "Contributors", "Entries"]   # Titles for columns in dataframe
for row in rows_headers:
    if row.find('td'):
        titles.append(row.find('td').text.rstrip())   # Getting title of each row 

In [14]:
# Main Dataframe for storing all the data
df = pd.DataFrame(columns = titles)
df

,City,Country,URL,Contributors,Entries,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),...,1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate"


In [21]:
# Dataframe for exceptions during scraping
df2 = pd.DataFrame(columns = titles)
df2

,City,Country,URL,Contributors,Entries,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),...,1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate"


In [15]:
# Creating a dictionary with country names as keys and list of cities names as values
cities_dict = {}
for country in countries:
    param["country"] = country
    page_country = requests.get(url_country, params=param)
    html = BeautifulSoup(page_country.text, 'html')
    cond = html.find('div', class_='align_like_price_table').text   # Information with number of entries and contributors
    if int(re.search(r'(\d+) entries', cond).group(1)) < 41 or int(re.search(r'(\d+) different contributors', cond).group(1)) < 6:
        #print('Too few data for', country)
        continue
    cities = html.find(id='city').find_all('option')
    cities_dict[country] = []
    for city in cities[1:]:
        cities_dict[country].append(city.text)   # Filling the dictionary

In [458]:
# The main part of the Scraping Data from numbeo
exceptions = {}    # dictionary for exceptions
cannot_find = {}   # dictionary for cities which URL are not found

# Function that searches for the right URL and returns it
def find_city(city_url):
    global found   # flag which shows that right URL was found and the next options do not have to be checked
    page_city = requests.get(url_city + city_url, params=param)
    html = BeautifulSoup(page_city.text, 'html')
    found = 1
    # If URL was specified incorrectly numbeo can show similar cities, so I check these cities if they have the same name
    # and the same country
    if re.match(r'Cannot find city id for', html.find('h1').text.lstrip()):
        found = 0
        cities_id = html.find('div', style="error_message").find_all('a')
        i = 0
        for city_id in cities_id:
            city_country = city_id.text.rsplit(', ', maxsplit=1)
            if city == city_country[0] and country == city_country[1]:
                link = html.find('div', style="error_message").find_all('a')[i]
                link_url = link.get('href')
                page_city = requests.get(link_url, params=param)
                found = 1
                break
            i = i + 1
    return page_city

for country in cities_dict.keys():
    for city in cities_dict[country]:
        # Different options for url that are stored in set, so there will be no duplicates
        city_country = city.replace(' ', '-').replace('(', '').replace(')', '').replace(',', '').replace('.', '') + '-' + country.replace(' ', '-')
        city_fixed = city.replace(' ', '-').replace('(', '').replace(')', '').replace(',', '').replace('.', '')
        city_state_country = re.sub(r',.+', '', city).replace(' ', '-').replace('(', '').replace(')', '').replace(',', '').replace('.', '') + '-' + country.replace(' ', '-')
        city_without_state = re.sub(r',.+', '', city).replace(' ', '-').replace('(', '').replace(')', '').replace(',', '').replace('.', '')
        city_without_bracket = re.sub(r'\(.+', '', city).rstrip()
        cities_url = {city_country, city_fixed, city_state_country, city_without_state, city_without_bracket}
        found = 0
        # Check each URL option in find_city() function
        for city_url in cities_url:
            page_city = find_city(city_url)
            html = BeautifulSoup(page_city.text, 'html')
            if found:
                break
        # If URL still cannot be found then I will just store those cities in cannot_find dictionary
        if re.match(r'Cannot find city id for', html.find('h1').text.lstrip()):
            if country in cannot_find.keys():
                cannot_find[country].append(city)
            else:
                cannot_find[country] = [city]
            print('Cannot find the city:', city)
            continue
        # Checking conditions if the data for that city was entered by more than 6 people and they made more than 41 entries
        # I created those conditions to have more reliable data
        cond = html.find('div', class_='align_like_price_table').text
        if re.search(r'(\d+) entries', cond) is None \
            or re.search(r'(\d+) different contributors', cond) is None \
            or int(re.search(r'(\d+) different contributors', cond).group(1)) < 6 \
            or int(re.search(r'(\d+) entries', cond).group(1)) < 41:
                print('Too few data for', city)
                continue
        entries = int(re.search(r'(\d+) entries', cond).group(1))
        contributors = int(re.search(r'(\d+) different contributors', cond).group(1))
        # Checking if the currency in USD
        currency = html.find('select', id="displayCurrency").find('option', selected="selected")
        if currency.text != param["displayCurrency"]:
            page_city = requests.get(page_city.url + "?displayCurrency=USD")
            html = BeautifulSoup(page_city.text, 'html')
        # Scraping all prices and converting them into float (if there is not price such entries will be filled with NaN)
        table = html.find_all('table')[1]
        rows_prices = table.find_all('tr')
        rows = [city, country, page_city.url, contributors, entries]
        for row in rows_prices:
            if row.find_all('td'):
                if row.find_all('td')[1].text.strip() == '?':
                    rows.append(np.nan)
                else:
                    rows.append(float(re.sub(r'[^0-9.]', '', row.find_all('td')[1].text)))
        length = len(df)
        try:
            df.loc[length] = rows
        except:
            if country in exceptions:
                exceptions[country].append(city)
            else:
                exceptions[country] = [city]
            continue

In [72]:
# Specific City Scraping
country = 'Ukraine'
city = 'Kiev'
exceptions = {}
page_city = requests.get('https://www.numbeo.com/cost-of-living/in/Kiev', params=param)
html = BeautifulSoup(page_city.text, 'html')
table = html.find_all('table')[1]
rows_prices = table.find_all('tr')
#print(rows_prices)
rows = [city, country]
for row in rows_prices:
    if row.find_all('td'):
        if row.find_all('td')[1].text.strip() == '?':
            rows.append(np.nan)
        else:
            rows.append(float(re.sub(r'[^0-9.]', '', row.find_all('td')[1].text)))
#print(rows)
length = len(df)
try:
    df.loc[length] = rows
except:
    if country in exceptions:
        exceptions[country].append(city)
    else:
        exceptions[country] = [city]

In [26]:
for country in exceptions:
    for city in exceptions[country]:
        print(country, city)

Bangladesh Cox's Bazar
Brunei Kuala Belait
India Gwalior
Indonesia Malang
Japan Saitama
Mauritius Grand Bay
Spain Almeria


In [20]:
i = 0
for country in cannot_find:
    for city in cannot_find[country]:
        i = i + 1
        
print(i)   # 0

0


In [29]:
for country in exceptions:
    for city in exceptions[country]:
        city_fixed = city.replace(' ', '-').replace('(', '').replace(')', '').replace(',', '')
        flag = 1
        page_city = requests.get(url_city + city_fixed + '-' + country.replace(' ', '-'), params=param)
        html = BeautifulSoup(page_city.text, 'html')
        if re.match(r'Cannot find city id for', html.find('h1').text.lstrip()):
            cities_id = html.find('div', style="error_message").find_all('a')
            i = 0
            for city_id in cities_id:
                city_country = city_id.text.rsplit(', ', maxsplit=1)
                if city == city_country[0] and country == city_country[1]:
                    link = html.find('div', style="error_message").find_all('a')[i]
                    link_url = link.get('href')
                    page_city = requests.get(link_url, params=param)
                    html = BeautifulSoup(page_city.text, 'html')
                    flag = 0
                    break
                i = i + 1
            if flag:
                page_city = requests.get(url_city + city_fixed, params=param)
                html = BeautifulSoup(page_city.text, 'html')
                if re.match(r'Cannot find city id for', html.find('h1').text.lstrip()):
                    print('Cannot find the city:', city)
                    continue
        cond = html.find('div', class_='align_like_price_table').text
        entries = int(re.search(r'(\d+) entries', cond).group(1))
        contributors = int(re.search(r'(\d+) different contributors', cond).group(1))
        currency = html.find('select', id="displayCurrency").find('option', selected="selected")
        if currency.text != param["displayCurrency"]:
            page_city = requests.get(page_city.url + "?displayCurrency=USD")
            html = BeautifulSoup(page_city.text, 'html')
        table = html.find_all('table')[2]
        rows_prices = table.find_all('tr')
        rows = [city, country, page_city.url, contributors, entries]
        for row in rows_prices:
            if row.find_all('td'):
                if row.find_all('td')[1].text.strip() == '?':
                    rows.append(np.nan)
                else:
                    rows.append(float(re.sub(r'[^0-9.]', '', row.find_all('td')[1].text)))
        length = len(df2)
        try:
            df2.loc[length] = rows
        except:
            continue

In [33]:
df.info()   # Checking if all types are correct

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, 0 to 1000
Data columns (total 60 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   City                                                                        1001 non-null   object 
 1   Country                                                                     1001 non-null   object 
 2   URL                                                                         1001 non-null   object 
 3   Contributors                                                                1001 non-null   int64  
 4   Entries                                                                     1001 non-null   int64  
 5   Meal, Inexpensive Restaurant                                                994 non-null    float64
 6   Meal for 2 People, Mid-range Restaurant, Three-course

In [30]:
df2

,City,Country,URL,Contributors,Entries,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),...,1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate"
0,Cox's Bazar,Bangladesh,https://www.numbeo.com/cost-of-living/in/Cox's...,6,79,1.82,20.50,3.42,9.11,2.73,...,36.45,91.13,182.27,109.36,341.75,182.27,515.00,367.86,318.97,NaN
1,Kuala Belait,Brunei,https://www.numbeo.com/cost-of-living/in/Kuala...,6,50,7.50,45.00,7.24,2.81,2.25,...,118.03,132.62,637.47,599.97,1050.64,900.79,NaN,NaN,2037.61,NaN
2,Gwalior,India,https://www.numbeo.com/cost-of-living/in/Gwali...,9,203,1.81,18.10,3.62,1.87,3.02,...,49.77,49.77,120.65,80.43,217.17,144.78,670.81,373.46,482.59,9.62
3,Malang,Indonesia,https://www.numbeo.com/cost-of-living/in/Malan...,9,92,1.38,7.70,3.21,2.89,4.65,...,54.54,77.00,170.04,48.12,288.74,96.25,NaN,NaN,273.77,4.00
4,Saitama,Japan,https://www.numbeo.com/cost-of-living/in/Saita...,7,47,9.63,28.73,4.23,5.07,3.38,...,52.39,101.41,405.62,236.61,1216.87,980.26,NaN,NaN,1780.24,1.86
5,Grand Bay,Mauritius,https://www.numbeo.com/cost-of-living/in/Grand...,10,110,9.81,50.12,4.63,3.27,5.45,...,116.22,83.53,617.42,363.19,958.81,1198.52,NaN,NaN,980.61,6.50
6,Almeria,Spain,https://www.numbeo.com/cost-of-living/in/Almer...,15,138,15.18,54.70,8.75,3.56,3.83,...,83.42,114.87,437.59,337.31,867.88,667.32,NaN,1312.77,1238.92,3.50


In [31]:
combined_df = pd.concat([df, df2])
combined_df.reset_index(drop=True, inplace=True)
combined_df.index = range(1, len(combined_df)+1)
combined_df

,City,Country,URL,Contributors,Entries,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),...,1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate"
1,Kabul,Afghanistan,https://www.numbeo.com/cost-of-living/in/Kabul...,14,149,2.12,11.30,5.30,4.94,1.69,...,44.73,32.49,139.23,95.34,261.31,171.26,434.54,264.84,181.86,14.25
2,Tirana,Albania,https://www.numbeo.com/cost-of-living/in/Tiran...,145,1867,10.56,52.78,7.92,2.64,3.69,...,92.78,123.28,545.12,369.81,1063.95,629.26,2761.26,1419.56,650.47,5.47
3,Vlore,Albania,https://www.numbeo.com/cost-of-living/in/Vlore...,25,285,8.44,31.67,6.33,3.17,3.43,...,65.97,52.78,330.77,233.32,493.03,381.04,1166.78,791.44,420.38,4.00
4,Algiers,Algeria,https://www.numbeo.com/cost-of-living/in/Algie...,127,1485,2.98,22.33,4.47,1.86,2.42,...,78.16,75.18,237.19,144.85,395.88,254.45,1995.27,1121.93,319.30,6.49
5,Annaba,Algeria,https://www.numbeo.com/cost-of-living/in/Annab...,10,134,2.98,22.33,4.47,0.89,1.49,...,83.87,81.88,200.99,129.03,310.16,210.91,953.45,545.92,282.87,5.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,Gwalior,India,https://www.numbeo.com/cost-of-living/in/Gwali...,9,203,1.81,18.10,3.62,1.87,3.02,...,49.77,49.77,120.65,80.43,217.17,144.78,670.81,373.46,482.59,9.62
1005,Malang,Indonesia,https://www.numbeo.com/cost-of-living/in/Malan...,9,92,1.38,7.70,3.21,2.89,4.65,...,54.54,77.00,170.04,48.12,288.74,96.25,NaN,NaN,273.77,4.00
1006,Saitama,Japan,https://www.numbeo.com/cost-of-living/in/Saita...,7,47,9.63,28.73,4.23,5.07,3.38,...,52.39,101.41,405.62,236.61,1216.87,980.26,NaN,NaN,1780.24,1.86
1007,Grand Bay,Mauritius,https://www.numbeo.com/cost-of-living/in/Grand...,10,110,9.81,50.12,4.63,3.27,5.45,...,116.22,83.53,617.42,363.19,958.81,1198.52,NaN,NaN,980.61,6.50


In [461]:
# df_result.index = range(1, len(df_result)+1)
df_result.to_csv(r'C:\Users\User\OneDrive\Documents\Python Web Scraping\numbeo_cities.csv')

In [34]:
df_new = combined_df
df_new

,City,Country,URL,Contributors,Entries,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),...,1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate"
1,Kabul,Afghanistan,https://www.numbeo.com/cost-of-living/in/Kabul...,14,149,2.12,11.30,5.30,4.94,1.69,...,44.73,32.49,139.23,95.34,261.31,171.26,434.54,264.84,181.86,14.25
2,Tirana,Albania,https://www.numbeo.com/cost-of-living/in/Tiran...,145,1867,10.56,52.78,7.92,2.64,3.69,...,92.78,123.28,545.12,369.81,1063.95,629.26,2761.26,1419.56,650.47,5.47
3,Vlore,Albania,https://www.numbeo.com/cost-of-living/in/Vlore...,25,285,8.44,31.67,6.33,3.17,3.43,...,65.97,52.78,330.77,233.32,493.03,381.04,1166.78,791.44,420.38,4.00
4,Algiers,Algeria,https://www.numbeo.com/cost-of-living/in/Algie...,127,1485,2.98,22.33,4.47,1.86,2.42,...,78.16,75.18,237.19,144.85,395.88,254.45,1995.27,1121.93,319.30,6.49
5,Annaba,Algeria,https://www.numbeo.com/cost-of-living/in/Annab...,10,134,2.98,22.33,4.47,0.89,1.49,...,83.87,81.88,200.99,129.03,310.16,210.91,953.45,545.92,282.87,5.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,Gwalior,India,https://www.numbeo.com/cost-of-living/in/Gwali...,9,203,1.81,18.10,3.62,1.87,3.02,...,49.77,49.77,120.65,80.43,217.17,144.78,670.81,373.46,482.59,9.62
1005,Malang,Indonesia,https://www.numbeo.com/cost-of-living/in/Malan...,9,92,1.38,7.70,3.21,2.89,4.65,...,54.54,77.00,170.04,48.12,288.74,96.25,NaN,NaN,273.77,4.00
1006,Saitama,Japan,https://www.numbeo.com/cost-of-living/in/Saita...,7,47,9.63,28.73,4.23,5.07,3.38,...,52.39,101.41,405.62,236.61,1216.87,980.26,NaN,NaN,1780.24,1.86
1007,Grand Bay,Mauritius,https://www.numbeo.com/cost-of-living/in/Grand...,10,110,9.81,50.12,4.63,3.27,5.45,...,116.22,83.53,617.42,363.19,958.81,1198.52,NaN,NaN,980.61,6.50


In [36]:
# Have more cities because initially Scraping algorithm was not so precise
df_old = pd.read_csv(r"C:\Users\User\OneDrive\Documents\Python Web Scraping\numbeo_cities_old.csv")
df_old

,Unnamed: 0,City,Country,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),Cappuccino (regular),Coke/Pepsi (0.33 liter bottle),...,1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate"
0,1,Kabul,Afghanistan,2.11,11.25,5.27,4.92,1.69,1.34,0.39,...,44.52,32.34,138.59,94.91,260.12,170.48,432.55,263.63,181.03,14.25
1,2,Tirana,Albania,10.53,52.66,7.90,2.63,3.69,1.80,1.56,...,92.10,122.89,537.14,367.39,1050.18,606.66,2746.42,1406.69,638.44,5.39
2,3,Vlore,Albania,8.43,31.60,6.32,3.16,3.42,1.66,1.65,...,65.82,52.66,330.22,232.93,492.18,380.38,1165.17,790.44,419.97,4.00
3,4,Algiers,Algeria,2.98,22.33,4.47,1.86,2.42,0.90,0.53,...,78.91,75.53,237.22,144.86,395.93,254.48,2003.16,1122.43,320.59,6.53
4,5,Annaba,Algeria,2.98,22.33,4.47,0.89,1.49,1.36,0.57,...,83.88,81.89,201.01,129.04,310.20,210.94,953.03,545.60,260.57,5.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,1089,Hanoi,Vietnam,2.03,24.36,4.87,1.01,1.79,1.94,0.45,...,87.86,100.36,351.97,214.94,917.08,587.35,2519.45,1401.97,498.62,11.22
1089,1090,Ho Chi Minh City,Vietnam,2.03,24.36,4.87,1.10,1.91,1.97,0.53,...,104.21,91.69,511.90,304.48,1119.77,596.14,4510.17,1990.79,461.49,10.06
1090,1091,Nha Trang,Vietnam,3.65,14.21,4.87,1.01,1.83,1.87,0.49,...,101.50,128.57,330.89,209.77,676.67,331.57,1624.00,943.07,350.17,11.50
1091,1092,Lusaka,Zambia,3.84,28.34,3.50,1.02,1.53,2.27,1.04,...,90.18,87.89,262.56,159.59,766.05,372.61,2000.00,4840.62,330.64,21.25


In [43]:
len(set(df_old["City"] + ', ' + df_old["Country"]))

1093

In [46]:
len(set(df_new["City"] + ', ' + df_new["Country"]))

1008

In [463]:
cities_countries = set(df_old["City"] + ', ' + df_old["Country"]) - set(df_new["City"] + ', ' + df_new["Country"])

In [ ]:
# Decided to add three more columns with population, latitude and longitude
# Downloaded different datasets and used APIs with city names and coordinates and tried to concatenate them with my DataFrame
# Then, found out that there is very helpful library Geopy for finding coordinates of locations and used it for left cities

In [70]:
df_country_code = pd.read_csv(r"C:\Users\User\Desktop\all.csv")
df_country_code

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN
245,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,NaN,2.0,15.0,NaN
246,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN
247,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0


In [295]:
city_country_df = df_result[df_result['lat'].isna()][['City', 'Country']]

In [353]:
pd.set_option('display.max_rows', 300)
df_country_code.loc[170, 'name'] = 'Palestine'

In [454]:
df_result['population'].replace(0, None, inplace=True)

In [455]:
df_result

,City,Country,URL,Contributors,Entries,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),...,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",population,lat,lng
0,Kabul,Afghanistan,https://www.numbeo.com/cost-of-living/in/Kabul...,14,149,2.12,11.30,5.30,4.94,1.69,...,95.34,261.31,171.26,434.54,264.84,181.86,14.25,4273156,34.5253,69.1783
1,Tirana,Albania,https://www.numbeo.com/cost-of-living/in/Tiran...,145,1867,10.56,52.78,7.92,2.64,3.69,...,369.81,1063.95,629.26,2761.26,1419.56,650.47,5.47,418495,41.3289,19.8178
2,Vlore,Albania,https://www.numbeo.com/cost-of-living/in/Vlore...,25,285,8.44,31.67,6.33,3.17,3.43,...,233.32,493.03,381.04,1166.78,791.44,420.38,4.00,130827,40.4667,19.4833
3,Algiers,Algeria,https://www.numbeo.com/cost-of-living/in/Algie...,127,1485,2.98,22.33,4.47,1.86,2.42,...,144.85,395.88,254.45,1995.27,1121.93,319.30,6.49,3415811,36.7539,3.0589
4,Annaba,Algeria,https://www.numbeo.com/cost-of-living/in/Annab...,10,134,2.98,22.33,4.47,0.89,1.49,...,129.03,310.16,210.91,953.45,545.92,282.87,5.75,257359,36.9,7.7667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,Gwalior,India,https://www.numbeo.com/cost-of-living/in/Gwali...,9,203,1.81,18.10,3.62,1.87,3.02,...,80.43,217.17,144.78,670.81,373.46,482.59,9.62,1069276,26.2215,78.178
1003,Malang,Indonesia,https://www.numbeo.com/cost-of-living/in/Malan...,9,92,1.38,7.70,3.21,2.89,4.65,...,48.12,288.74,96.25,NaN,NaN,273.77,4.00,2795209,-7.98,112.62
1004,Saitama,Japan,https://www.numbeo.com/cost-of-living/in/Saita...,7,47,9.63,28.73,4.23,5.07,3.38,...,236.61,1216.87,980.26,NaN,NaN,1780.24,1.86,1325843,35.8614,139.6456
1005,Grand Bay,Mauritius,https://www.numbeo.com/cost-of-living/in/Grand...,10,110,9.81,50.12,4.63,3.27,5.45,...,363.19,958.81,1198.52,NaN,NaN,980.61,6.50,None,-20.01302,57.584627


In [438]:
df_result.loc[41, ['population', 'lat', 'lng']] = [114486.0, 36.8625, 10.195556]

In [447]:
df_result['population'] = df_result['population'].fillna(0).astype(int)

In [429]:
!pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --------- ------------------------------ 30.7/125.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [468]:
from geopy.geocoders import Nominatim

cannot_find = {}
for index, row in df_result[df_result['lat'].isna()][['City', 'Country']].iterrows():
    geolocator = Nominatim(user_agent="myapp")
    location = geolocator.geocode(row[0] + ', ' + row[1])
    print(row[0], row[1])
    try:
        print(location.latitude, location.longitude)
        df_result.loc[index, ['lat', 'lng']] = [location.latitude, location.longitude]
    except:
        if row[1] in cannot_find.keys():
            cannot_find[row[1]].append(row[0])
        else:
            cannot_find[row[1]] = [row[0]]
        continue

In [435]:
print(cannot_find)

{'Tunisia': ['Aryanah (Ariana)']}


In [467]:
cannot_find = {}
for index, row in city_country_df.iterrows():
    name = row[0].split(', ')[0].split(' (')[0]
    region = df_country_code[df_country_code['name'] == row[1]]['region'].to_string(index=False)
    iso_code = df_country_code[df_country_code['name'].str.lower() == row[1].lower()]['alpha-2'].to_string(index=False)
    country = iso_code
    api_url = 'https://api.api-ninjas.com/v1/city?name={}&country={}'.format(name, country)
    response = requests.get(api_url, headers={'X-Api-Key': '02J1tSOXSToXbbBaEhAakQ==htau0A7BXfbyur2Q'})
    if response.status_code == requests.codes.ok:
        if len(response.text) > 3:
            response_dict = json.loads(response.text[1:-1])
            print(float(response_dict['population']), response_dict['latitude'], response_dict['longitude'])
            df_result.loc[index, ['population', 'lat', 'lng']] = [response_dict['population'], response_dict['latitude'], response_dict['longitude']] 
        else:
            if row[1] in cannot_find.keys():
                cannot_find[row[1]].append(row[0])
            else:
                cannot_find[row[1]] = [row[0]]
    else:
        print("Error:", response.status_code, response.text)

In [132]:
df_cities = pd.read_csv(r"C:\Users\User\Desktop\worldcities.csv")
df_add = df_cities[['city_ascii', 'country', 'population', 'lat', 'lng']]

In [464]:
# pd.set_option('display.max_rows', 10)
# df_result = df_new.merge(df_add, how = 'left')
# df_result

In [135]:
df_add = df_add.rename(columns = {'city_ascii': 'City', 'country': 'Country'})

In [227]:
df_result

,City,Country,URL,Contributors,Entries,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Non-Alcoholic Beer (0.5 liter draught),Imported Non-Alcoholic Beer (0.33 liter bottle),...,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",population,lat,lng
0,Kabul,Afghanistan,https://www.numbeo.com/cost-of-living/in/Kabul...,14,149,2.12,11.30,5.30,4.94,1.69,...,95.34,261.31,171.26,434.54,264.84,181.86,14.25,4273156.0,34.5253,69.1783
1,Tirana,Albania,https://www.numbeo.com/cost-of-living/in/Tiran...,145,1867,10.56,52.78,7.92,2.64,3.69,...,369.81,1063.95,629.26,2761.26,1419.56,650.47,5.47,418495.0,41.3289,19.8178
2,Vlore,Albania,https://www.numbeo.com/cost-of-living/in/Vlore...,25,285,8.44,31.67,6.33,3.17,3.43,...,233.32,493.03,381.04,1166.78,791.44,420.38,4.00,130827.0,40.4667,19.4833
3,Algiers,Algeria,https://www.numbeo.com/cost-of-living/in/Algie...,127,1485,2.98,22.33,4.47,1.86,2.42,...,144.85,395.88,254.45,1995.27,1121.93,319.30,6.49,3415811.0,36.7539,3.0589
4,Annaba,Algeria,https://www.numbeo.com/cost-of-living/in/Annab...,10,134,2.98,22.33,4.47,0.89,1.49,...,129.03,310.16,210.91,953.45,545.92,282.87,5.75,257359.0,36.9000,7.7667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,Gwalior,India,https://www.numbeo.com/cost-of-living/in/Gwali...,9,203,1.81,18.10,3.62,1.87,3.02,...,80.43,217.17,144.78,670.81,373.46,482.59,9.62,1069276.0,26.2215,78.1780
1004,Malang,Indonesia,https://www.numbeo.com/cost-of-living/in/Malan...,9,92,1.38,7.70,3.21,2.89,4.65,...,48.12,288.74,96.25,NaN,NaN,273.77,4.00,2795209.0,-7.9800,112.6200
1005,Saitama,Japan,https://www.numbeo.com/cost-of-living/in/Saita...,7,47,9.63,28.73,4.23,5.07,3.38,...,236.61,1216.87,980.26,NaN,NaN,1780.24,1.86,1325843.0,35.8614,139.6456
1006,Grand Bay,Mauritius,https://www.numbeo.com/cost-of-living/in/Grand...,10,110,9.81,50.12,4.63,3.27,5.45,...,363.19,958.81,1198.52,NaN,NaN,980.61,6.50,NaN,NaN,NaN


In [207]:
df_add

,City,Country,population,lat,lng
0,Tokyo,Japan,37732000.0,35.6897,139.6922
1,Jakarta,Indonesia,33756000.0,-6.1750,106.8275
2,Delhi,India,32226000.0,28.6100,77.2300
3,Guangzhou,China,26940000.0,23.1300,113.2600
4,Mumbai,India,24973000.0,19.0761,72.8775
...,...,...,...,...,...
44686,Numto,Russia,10.0,63.6667,71.3333
44687,Nord,Greenland,10.0,81.7166,-17.8000
44688,Timmiarmiut,Greenland,10.0,62.5333,-42.2167
44689,San Rafael,Bolivia,NaN,-16.7795,-60.6799


In [466]:
city_country_df = df_result[df_result['lat'].isna()][["City", "Country"]]
for index, row in city_country_df.iterrows():
    city = row[0].split(', ')[0].split(' (')[0]
    country = row[1]
    row_found = df_add_geo[((df_add_geo['ASCII Name'] == city) | (df_add_geo['Name'] == city) | (city in df_add_geo['Alternate Names'])) & (df_add_geo['Country name EN'] == country)]
    if not row_found.empty:
        index = df_result[(df_result['City'] == row[0]) & (df_result['Country'] == row[1])].index.values[0]
        pop = row_found['Population'].values[0]
        lat = row_found['lat'].values[0]
        lng = row_found['lng'].values[0]
        print(index, pop, lat, lng)
        df_result.loc[index, ['population', 'lat', 'lng']] = [pop, lat, lng]